In [5]:
# CELL 1: Imports
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [6]:
# CELL 2: Data Setup & Transforms (Using your existing paths)
dataset_root = r"C:\Users\yozev\OneDrive\Desktop\artFiltered"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

full_dataset = ImageFolder(root=dataset_root, transform=transform)
print("Classes:", full_dataset.classes)

Classes: ['Abstract_Expressionism', 'Art_Nouveau_Modern', 'Baroque', 'Cubism', 'Expressionism', 'Impressionism', 'Naive_Art_Primitivism', 'Northern_Renaissance', 'Post_Impressionism', 'Realism', 'Rococo', 'Romanticism', 'Symbolism']


In [7]:
# CELL 3: Data Splitting
dataset_size = len(full_dataset)
train_size = int(0.75 * dataset_size)
val_size = int(0.10 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [8]:
class HyperNetwork(nn.Module):
    def __init__(self, z_dim, target_hidden_dim):
        super(HyperNetwork, self).__init__()

        self.embedding = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )

        self.weight_generator = nn.Sequential(
            nn.Linear(256, 2048 * 512),
            nn.Tanh()
        )

        self.bias_generator = nn.Sequential(
            nn.Linear(256, 512),
            nn.Tanh()
        )

    def forward(self, z):
        embedded = self.embedding(z)
        weights = self.weight_generator(embedded)
        weights = weights.view(2048, 512)
        biases = self.bias_generator(embedded)
        return weights, biases

class HyperResNet(nn.Module):
    def __init__(self, num_classes, z_dim=64):
        super(HyperResNet, self).__init__()

        self.resnet = models.resnet50(weights='IMAGENET1K_V2')
        self.feature_extractor = nn.Sequential(*list(self.resnet.children())[:-1])

        # Fixed dimensions for clarity
        self.input_dim = 2048  # ResNet50 output dimension
        self.hidden_dim = 512  # Hidden dimension

        self.hyper_net = HyperNetwork(z_dim, self.hidden_dim)
        self.z_dim = z_dim
        self.z = nn.Parameter(torch.randn(z_dim))

        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # Get 2048-dimensional features from ResNet
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)  # Batch x 2048

        # Generate weights (2048 x 512) and biases (512)
        weights, biases = self.hyper_net(self.z)

        # Matrix multiplication: (Batch x 2048) @ (2048 x 512) = (Batch x 512)
        x = torch.matmul(features, weights) + biases
        x = torch.relu(x)

        # Final classification
        x = self.classifier(x)
        return x

In [9]:
# CELL 5: Model Setup and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize model
num_classes = len(full_dataset.classes)
model = HyperResNet(num_classes=num_classes)  # Make sure to instantiate the model
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

# Create parameter groups, excluding z from the main parameters
base_params = [p for n, p in model.named_parameters() if n != 'z']
optimizer = optim.Adam([
    {'params': base_params, 'lr': 1e-4},
    {'params': [model.z], 'lr': 1e-3}
])

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=3,
    verbose=True
)

Using device: cuda


C:\Users\yozev\AppData\Roaming\Python\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [20]:
try:
    print("Training started...")
    for epoch in range(epochs):
        # Training
        model.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{epochs}] Training')

        for batch_idx, (images, labels) in enumerate(train_bar):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_bar.set_postfix({
                'batch': f'{batch_idx+1}/{len(train_loader)}',
                'train_loss': f'{loss.item():.4f}'
            })

        avg_train_loss = running_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f'Epoch [{epoch+1}/{epochs}] Validation')
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        accuracy = 100 * correct / total

        # Metrics
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')

        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'z_vector': model.z.data,
                'best_accuracy': best_accuracy,
            }, os.path.join(dataset_root, 'best_hyper_model.pth'))
            patience_counter = 0
        else:
            patience_counter += 1

        # Print summary
        print(f"\nEpoch [{epoch+1}/{epochs}] Summary:")
        print(f"Training Loss: {avg_train_loss:.4f}")
        print(f"Validation Loss: {avg_val_loss:.4f}")
        print(f"Validation Accuracy: {accuracy:.2f}%")
        print(f"Validation Precision: {precision:.4f}")
        print(f"Validation Recall: {recall:.4f}")
        print(f"Best Accuracy: {best_accuracy:.2f}%")
        print('-'*50)

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

        scheduler.step(avg_val_loss)

except KeyboardInterrupt:
    print("\nTraining interrupted. Saving the current model...")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'z_vector': model.z.data,
        'best_accuracy': best_accuracy,
    }, os.path.join(dataset_root, 'interrupted_model.pth'))
    print("Model saved as 'interrupted_model.pth'. You can resume or test it later.")

print("\nTraining completed!")
print(f"Best validation accuracy: {best_accuracy:.2f}%")


Training started...


Epoch [1/20] Validation: 100%|██████████| 203/203 [01:27<00:00,  2.31it/s]



Epoch [1/20] Summary:
Training Loss: 1.3295
Validation Loss: 1.0151
Validation Accuracy: 65.68%
Validation Precision: 0.6692
Validation Recall: 0.6568
Best Accuracy: 65.68%
--------------------------------------------------


Epoch [2/20] Validation: 100%|██████████| 203/203 [01:25<00:00,  2.38it/s]



Epoch [2/20] Summary:
Training Loss: 0.8020
Validation Loss: 0.9324
Validation Accuracy: 68.57%
Validation Precision: 0.7017
Validation Recall: 0.6857
Best Accuracy: 68.57%
--------------------------------------------------


...


Epoch [10/20] Summary:
Training Loss: 0.0286
Validation Loss: 1.5207
Validation Accuracy: 72.39%
Validation Precision: 0.7250
Validation Recall: 0.7239
Best Accuracy: 72.39%
--------------------------------------------------


Epoch [11/20] Validation: 100%|██████████| 203/203 [01:26<00:00,  2.35it/s]



Epoch [11/20] Summary:
Training Loss: 0.0189
Validation Loss: 1.5783
Validation Accuracy: 72.11%
Validation Precision: 0.7220
Validation Recall: 0.7211
Best Accuracy: 72.39%
--------------------------------------------------


Epoch [12/20] Training:   4%|▍         | 61/1518 [06:25<2:33:24,  6.32s/it, batch=61/1518, train_loss=0.0004]


KeyboardInterrupt: 

In [10]:
checkpoint = torch.load(os.path.join(dataset_root, 'best_hyper_model.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

test_loss = 0.0
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc='Testing'):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = 100 * correct / total
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')

print("\nTest Set Results:")
print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

C:\Users\yozev\AppData\Local\Temp\ipykernel_41648\1891230433.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(dataset_root, 'best_hy


Test Set Results:
Test Accuracy: 71.69%
Test Precision: 0.7167
Test Recall: 0.7169
